# Explore here

It's recommended to use this notebook for exploration purposes.

For example: 

1. You could import the CSV generated by python into your notebook and explore it.
2. You could connect to your database using `pandas.read_sql` from this notebook and explore it.

In [32]:
!pip install -r ../requirements.txt

In [33]:
!pip install seaborn
!pip install --upgrade pip

!pip install numpy 
!pip install Flask 
!pip install request
!pip install render_template pickle

ERROR: Could not find a version that satisfies the requirement request (from versions: none)
ERROR: No matching distribution found for request
ERROR: Could not find a version that satisfies the requirement render_template (from versions: none)
ERROR: No matching distribution found for render_template


In [34]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

from flask import Flask, request, render_template
import pickle

In [35]:
url = "https://raw.githubusercontent.com/4GeeksAcademy/random-forest-project-tutorial/main/titanic_train.csv"
df = pd.read_csv(url)

In [36]:
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
416,417,1,2,"Drew, Mrs. James Vivian (Lulu Thorne Christian)",female,34.0,1,1,28220,32.5000,NaN,S
782,783,0,1,"Long, Mr. Milton Clyde",male,29.0,0,0,113501,30.0000,D6,S
858,859,1,3,"Baclini, Mrs. Solomon (Latifa Qurban)",female,24.0,0,3,2666,19.2583,NaN,C
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S
637,638,0,2,"Collyer, Mr. Harvey",male,31.0,1,1,C.A. 31921,26.2500,NaN,S


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [38]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [39]:
df["Survived"].value_counts()
#la muestra no esta desbalanceada

0    549
1    342
Name: Survived, dtype: int64

In [40]:
#df =df.drop(['Cabin','PassengerId','Ticket','Name',],axis=1)
df=df.drop(columns=["Name"])
#No tienen valor predictivo la elimino

In [41]:
df["Sex"]=df["Sex"].map({"male":1,"female":0})
df["Sex"]

0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: Sex, Length: 891, dtype: int64

In [42]:
df["Ticket"].nunique()

681

In [43]:
df=df.drop(columns=["Ticket"])
# no aporta a la predicción

In [44]:
df=df.drop(columns=["Cabin"])
df=df.drop(columns=["PassengerId"])
df=df.drop(columns=["SibSp"])

# no aporta a la predicción

In [45]:
df["Embarked"].value_counts()
df["Embarked"].nunique()

3

In [46]:
df["Embarked"]=df["Embarked"].map({"S":2,"C":1,"Q":0})
df["Embarked"]

0      2.0
1      1.0
2      2.0
3      2.0
4      2.0
      ... 
886    2.0
887    2.0
888    2.0
889    1.0
890    0.0
Name: Embarked, Length: 891, dtype: float64

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       714 non-null    float64
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  889 non-null    float64
dtypes: float64(3), int64(4)
memory usage: 48.9 KB


In [48]:
df["Age"].mean()
# analizo las edades

29.69911764705882

In [61]:
#df["Age"][np.isnan(df["Age"])]=df["Age"].mean()
#sino tiene edad lo completo con la media

df['Age_clean']=df['Age'].fillna(29)
df=df.drop(['Age'],axis=1)

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Survived   891 non-null    int64  
 1   Pclass     891 non-null    int64  
 2   Sex        891 non-null    int64  
 3   Parch      891 non-null    int64  
 4   Fare       891 non-null    float64
 5   Embarked   891 non-null    float64
 6   Age_clean  891 non-null    float64
dtypes: float64(3), int64(4)
memory usage: 48.9 KB


In [66]:
df["Embarked"].value_counts()

2.0    646
1.0    168
0.0     77
Name: Embarked, dtype: int64

In [65]:
df.sample(5)

,Survived,Pclass,Sex,Parch,Fare,Embarked,Age_clean
668,0,3,1,0,8.0500,2.0,43.000000
451,0,3,1,0,19.9667,2.0,29.699118
635,1,2,0,0,13.0000,2.0,28.000000
767,0,3,0,0,7.7500,0.0,30.500000
646,0,3,1,0,7.8958,2.0,19.000000


In [51]:
df['Embarked'][np.isnan(df['Embarked'])]=2
# completo con 2 los valores en 0

/tmp/ipykernel_2288/151368785.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Embarked'][np.isnan(df['Embarked'])]=2


In [52]:
df.sample(5)

,Survived,Pclass,Sex,Age,Parch,Fare,Embarked
423,0,3,0,28.0,1,14.4000,2.0
479,1,3,0,2.0,1,12.2875,2.0
262,0,1,1,52.0,1,79.6500,2.0
690,1,1,1,31.0,0,57.0000,2.0
673,1,2,1,31.0,0,13.0000,2.0


In [53]:
X=df.drop(columns=['Survived'])
y = df[['Survived']]

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=520, test_size=0.2)

In [68]:
modelo = RandomForestClassifier(n_estimators=20, random_state=42, max_depth=5)

#Uso RandomForestClassifier

In [69]:
modelo.fit(X_train, y_train)

/tmp/ipykernel_2288/2554334259.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelo.fit(X_train, y_train)


RandomForestClassifier(max_depth=5, n_estimators=20, random_state=42)

In [56]:
y_train_pred=modelo.predict(X_train)
y_test_pred=modelo.predict(X_test)

In [57]:
print(metrics.classification_report(y_pred=y_train_pred, y_true=y_train))

              precision    recall  f1-score   support

           0       0.83      0.97      0.90       456
           1       0.92      0.66      0.77       256

    accuracy                           0.86       712
   macro avg       0.88      0.81      0.83       712
weighted avg       0.87      0.86      0.85       712



In [58]:
print(metrics.classification_report(y_pred=y_test_pred, y_true=y_test))

              precision    recall  f1-score   support

           0       0.70      0.97      0.81        93
           1       0.94      0.56      0.70        86

    accuracy                           0.77       179
   macro avg       0.82      0.76      0.76       179
weighted avg       0.82      0.77      0.76       179



In [59]:
filename = '../models/titanic_model.pickle'
pickle.dump(modelo, open(filename, 'wb'))

In [60]:
titanic_model = pickle.load(open('../models/titanic_model.pickle', 'rb'))